# Military Emissions

This notebook is responsible for working with data from the SIPRI Arms Transfer database.

The value column corresponds to the SIPRI TIV value. It is a measure of military capability transferred.

In [19]:
import pandas as pd

In [20]:
# Read the CSV file into a dataframe
df = pd.read_csv('data/trade-register.csv')
columns_to_drop = ['col4', 'Number ordered', 'col6', 'Weapon designation', 'Weapon description', 'Number delivered', 'col10', 'Year(s) of delivery', 'status', 'Comments', 'SIPRI TIV per unit', 'SIPRI TIV for total order']
df.drop(columns=columns_to_drop, inplace=True)
# Rename the columns
df.columns=['Recipient', 'Supplier', 'Year', 'sipri_TIV']
# I am recoding the Soviet Union as Russia because the Soviet Union no longer exists, and was a big player in our 1950-2023 dataset. Thus, maintaining that data is useful.
df['Supplier'] = df['Supplier'].replace('Soviet Union', 'Russian Federation')
df['Recipient'] = df['Recipient'].replace('Soviet Union', 'Russian Federation')
# Display the head of the dataframe
df.head()
print(len(df['Recipient']))

29058


In [21]:
from functions import country_to_iso

df['ISO_Recipient'] = df['Recipient'].apply(country_to_iso)
df['ISO_Supplier'] = df['Supplier'].apply(country_to_iso)

df.dropna(inplace=True)
df.head()
print(len(df['Recipient']))
unique_iso_recipients = df['ISO_Recipient'].unique()
print(unique_iso_recipients)

24705
['AFG' 'ALB' 'DZA' 'AGO' 'ATG' 'ARG' 'ARM' 'AUT' 'AUS' 'AZE' 'BHR' 'BGD'
 'BRB' 'BHS' 'BDI' 'BEL' 'BEN' 'BFA' 'BTN' 'BLR' 'BLZ' 'BOL' 'BWA' 'BRA'
 'BGR' 'CMR' 'CAN' 'CPV' 'CAF' 'TCD' 'CHL' 'CHN' 'KHM' 'COL' 'COM' 'COG'
 'CRI' 'HRV' 'CUB' 'CYP' 'CZE' 'DNK' 'DJI' 'DOM' 'ECU' 'EGY' 'GNQ' 'ERI'
 'EST' 'TLS' 'ETH' 'FIN' 'FJI' 'FRA' 'DEU' 'GAB' 'GMB' 'GNB' 'GEO' 'GHA'
 'GRD' 'GRC' 'GTM' 'GIN' 'GUY' 'HTI' 'HND' 'HUN' 'ISL' 'IND' 'IDN' 'IRN'
 'IRL' 'IRQ' 'ISR' 'ITA' 'JAM' 'JPN' 'JOR' 'KAZ' 'KEN' 'KIR' 'PRK' 'KOR'
 'KWT' 'KGZ' 'LAO' 'LVA' 'LBN' 'LSO' 'LBR' 'LTU' 'LUX' 'LBY' 'MKD' 'MDG'
 'MYS' 'MHL' 'MUS' 'MEX' 'MLI' 'MDV' 'MDA' 'MNG' 'MAR' 'MOZ' 'MRT' 'MLT'
 'MNE' 'MWI' 'MMR' 'NAM' 'NPL' 'NLD' 'NIC' 'NGA' 'NER' 'NOR' 'NZL' 'OMN'
 'PAK' 'PLW' 'PAN' 'PNG' 'PRY' 'PER' 'PHL' 'POL' 'PRT' 'QAT' 'ROU' 'RWA'
 'ZAF' 'ESH' 'SLV' 'WSM' 'SAU' 'SEN' 'SRB' 'SYC' 'SLE' 'SGP' 'KNA' 'SVK'
 'SVN' 'SLB' 'SOM' 'ESP' 'LKA' 'SSD' 'SDN' 'SUR' 'SWZ' 'SWE' 'CHE' 'SYR'
 'TWN' 'TJK' 'TZA' 'THA' 'TGO' 'TON' 'TTO' 'T

In [22]:
from functions import df_to_database

df_to_database(df, 'tradeRegister')

'Success.'

In [ ]:
year_counts = df['Year'].value_counts()
print(year_counts)

# Visualisations
This is mostly code that can then be transferred into Flask.

In [23]:
import geopandas as gpd

import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [24]:
fig = go.Figure()

df = df[df['Year'] == 2022]
df_zip = zip(df['ISO_Supplier'], df['ISO_Recipient'], df['sipri_TIV'])

for iso_supplier, iso_recipient, value in df_zip:
    fig.add_trace(go.Scattergeo(
                        locations=[iso_supplier, iso_recipient],
                        locationmode='ISO-3',
                        mode='lines',
                        line=dict(width=value/500, color="red")
                        ))

fig.update_layout(title="Arms transfers between countries",
                  margin={"t": 0, "b": 0, "l": 0, "r": 0, "pad": 0},
                  showlegend=False)
# fig.update_geos(projection_type="orthographic")

fig.show()